<a href="https://colab.research.google.com/github/alzimna/Anmul/blob/main/Chapters/Bab_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [182]:
import numpy as np
import pandas as pd
import scipy
import os
import math
import cv2
import shutil
import requests, zipfile, io
from PIL import Image
from skimage import feature, color, data
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from IPython.display import clear_output
from numpy.linalg import inv as inv
from numpy.linalg import eig as eig
from numpy import transpose as t
from numpy.linalg import det as det

# **Example 11.3**



In [183]:
url = 'https://github.com/alzimna/Anmul/raw/main/data/t10.1.xlsx'
data = pd.read_excel(url)
print(data.head())

   Obs    y1    y2    y3   x1    x2   x3
0    1  41.5  45.9  11.2  162  23.0  3.0
1    2  33.8  53.3  11.2  162  23.0  8.0
2    3  27.7  57.5  12.7  162  30.0  5.0
3    4  21.7  58.8  16.0  162  30.0  8.0
4    5  19.9  60.6  16.2  172  25.0  5.0


In [184]:
Sec = pd.DataFrame()
Sec["x1^2"] = data.iloc[:,4]**2
Sec["x2^2"] = data.iloc[:,5]**2
Sec["x3^2"] = data.iloc[:,6]**2
Sec["x1x2"] = data.iloc[:,4] * data.iloc[:,5]
Sec["x1x3"] = data.iloc[:,4] * data.iloc[:,6]
Sec["x2x3"] = data.iloc[:,5] * data.iloc[:,6]
databaru = pd.concat([data,Sec],axis = 1)

In [185]:
databaru.head()

,Obs,y1,y2,y3,x1,x2,x3,x1^2,x2^2,x3^2,x1x2,x1x3,x2x3
0,1,41.5,45.9,11.2,162,23.0,3.0,26244,529.0,9.0,3726.0,486.0,69.0
1,2,33.8,53.3,11.2,162,23.0,8.0,26244,529.0,64.0,3726.0,1296.0,184.0
2,3,27.7,57.5,12.7,162,30.0,5.0,26244,900.0,25.0,4860.0,810.0,150.0
3,4,21.7,58.8,16.0,162,30.0,8.0,26244,900.0,64.0,4860.0,1296.0,240.0
4,5,19.9,60.6,16.2,172,25.0,5.0,29584,625.0,25.0,4300.0,860.0,125.0


In [186]:
R = databaru.iloc[:,1:].corr()
print(round(R.iloc[:3,3:],2))

      x1    x2    x3  x1^2  x2^2  x3^2  x1x2  x1x3  x2x3
y1 -0.68 -0.22 -0.45 -0.68 -0.23 -0.42 -0.41 -0.55 -0.45
y2  0.40  0.08  0.39  0.40  0.12  0.33  0.16  0.44  0.33
y3  0.58  0.23  0.36  0.58  0.22  0.36  0.40  0.45  0.39


In [187]:
Ryy = R.iloc[:3,:3]
Ryx = R.iloc[:3,3:]
Rxy = R.iloc[3:,:3]
Rxx = R.iloc[3:,3:]

In [188]:
M = inv(Ryy) @ Ryx.values @ inv(Rxx) @ Rxy.values
r_square = eig(M)[0]
np.ndarray.sort(r_square)
r_square = r_square[::-1]
r = np.sqrt(r_square)
rel = r_square/sum(r_square)
df = pd.DataFrame({
    "r" : r,
    "r^2" :r_square,
    "Relative Size" : rel
}
)
df

,r,r^2,Relative Size
0,0.989931,0.979963,0.466277
1,0.952785,0.907799,0.431940
2,0.462510,0.213916,0.101783


In [189]:
#Find correlation for any y with x is given
r_x = []
for i in range(3) :
  ryx = R.iloc[i,3:]
  r_x.append(np.sqrt(t(ryx) @ inv(Rxx) @ ryx))
df = pd.DataFrame({
    "Var" : ["y1","y2","y3"],
    "Correlation" : np.round(r_x,3)
})
print(t(df))

                 0      1      2
Var             y1     y2     y3
Correlation  0.987  0.921  0.906


In [190]:
#Find correlation for any x with y is given
r_y = []
for i in range(9) :
  rxy = R.iloc[i+3,:3]
  r_y.append(np.sqrt(t(rxy) @ inv(Ryy) @ rxy))
df = pd.DataFrame({
    "Var" : ["x1","x2","x3","x1^2","x2^2","x3^2","x1x2","x1x3","x2x3"],
    "Correlation" : np.round(r_y,3)
})
print(t(df))

                 0      1      2     3      4      5      6      7      8
Var             x1     x2     x3  x1^2   x2^2   x3^2   x1x2   x1x3   x2x3
Correlation  0.691  0.237  0.507  0.69  0.234  0.466  0.431  0.585  0.482


# **Example 11.4.1**

In [191]:
# Lambda Wilks Test
Lambda = 1
for item in r_square :
  Lambda*=1-item
print("Lambda = ",Lambda)

# Pillai
V = 0
for item in r_square :
  V+=item
print("V = ",V)

# Lawley
U = 0
for item in r_square :
  U+=item/(1-item)
print("U = ",U)

# Roys test
theta = r_square[0]
print("Theta = ",theta)

Statistics = list([Lambda,V,U,theta])

Lambda =  0.001452217268617053
V =  2.101678066069881
U =  59.026402333076334
Theta =  0.9799633030032007


In [192]:
#Wilks
p = 3
q = 9
n = data.shape[0]
w = n-1/2*(p+q+3)
t = np.sqrt((p**2*q**2-4)/(p**2+q**2-5))
df1 = p*q
df2 = w*t-p*q/2+1

F_Lambda = (1-Lambda**(1/t))/Lambda**(1/t) * df2/df1

#Pillai
s = min(p,q)
d = max(p,q)
vE = 9

F_V = (vE-p+s)/d*V/(s-V)

#Lawley Hotelling
vH = 3
p = 9
vE = 9
s = 3
F_U = (s*(vE-vH-1)+2)/(s*p*vH)*U

#Roy
vE = 19
d = max(p,vH)
lambda1 = theta/(1-theta)
F_theta = (vE-d-1)/d*lambda1

F = list([F_Lambda,F_V,F_U,F_theta])

df = pd.DataFrame({
    "Statistics" : ["Wilks","Pillai","Lawley-Hotelling","Roy"],
    "Approximate F" : F,
    "df1" : list([27,27,27,9]),
    "df2" : list([21.09,27,27,9])
})
print(df)

         Statistics  Approximate F  df1    df2
0             Wilks       6.536567   27  21.09
1            Pillai       2.339560   27  27.00
2  Lawley-Hotelling      12.388257   27  27.00
3               Roy      48.908426    9   9.00


#**Example 11.4.2**


In [193]:
Lambda = [Lambda]
for i in range(1,3) :
  Lambda.append(round(Lambda[i-1]/(1-r_square[i-1]),5))
F = []
for k in range(1,4) :
  p = 3-k+1
  q = 9-k+1
  n = data.shape[0]
  w = n-1/2*(3+9+3)
  t = np.sqrt((p**2*q**2-4)/(p**2+q**2-5))
  df1 = p*q
  df2 = w*t-p*q/2+1
  Fk = (1-Lambda[k-1]**(1/t))/Lambda[k-1]**(1/t) * df2/df1
  F.append(round(Fk,3))
df = pd.DataFrame({
    "k" : [1,2,3],
    "Lambda_k" : Lambda,
    "Lambda_0.05" : [0.024,0.069,0.209],
    "Approximate F" : F,
    "df1" : list([27,16,7]),
    "df2" : list([21.1,16,9])
})
print(df)


   k  Lambda_k  Lambda_0.05  Approximate F  df1   df2
0  1  0.001452        0.024          6.537   27  21.1
1  2  0.072480        0.069          2.714   16  16.0
2  3  0.786110        0.209          0.350    7   9.0


#**Example 11.5.1**

In [194]:
M_c = inv(Ryy) @ Ryx.values @ inv(Rxx) @ Rxy.values
ev_c = eig(M_c)[1]

M_d = inv(Rxx) @ Rxy.values @ inv(Ryy) @ Ryx.values
ev_d = eig(M_d)[1].real

In [195]:
l = [1.536/-0.94851541,4.4704/-0.72720025,5.7961/-0.71879743]
c1 = np.round(ev_c[:,1]*l[0],4)
c2 = np.round(ev_c[:,2]*l[1],4)
c3 = np.round(ev_c[:,0]*l[2],4)
df = pd.DataFrame({
    "c1" : c1,
    "c2" : c2,
    "c3" : c3
},index = ["y1","y2","y3"])
print(df)

        c1      c2      c3
y1  1.5360  4.4704  5.7961
y2  0.2108  2.8291  2.2281
y3  0.4676  3.1309  5.1442


In [196]:
k = [5.0125/0.52254263,-38.3053/0.66212816,-12.5072/-0.22553688]
d1 = np.round(ev_d[:,0]*k[0],4)
d2 = np.round(ev_d[:,1]*k[1],4)
d3 = np.round(ev_d[:,2]*k[2],4)
df = pd.DataFrame({
    "d1" : d1,
    "d2" : d2,
    "d3" : d3
},index = ["x1","x2","x3","x1^2","x2^2","x3^2","x1x2","x1x3","x2x3"])
print(df)

          d1       d2       d3
x1    5.0125 -38.3053 -12.5072
x2    5.8551 -17.7390 -24.2291
x3    1.6500  -7.9699 -32.7393
x1^2 -2.6655  32.7933   4.8454
x2^2 -1.2346  -3.3641  10.7979
x3^2  0.5703   0.8733   0.9706
x1x2 -3.9209  19.2937  11.9421
x1x3 -2.2968   6.4001  31.2190
x2x3  0.5316   0.8096   1.2988
